In [ ]:
'''
This script is intended for combinig values from xml_synsets and xml_relations tables into
clear relations table 'relations_complete.db'
'''

In [52]:
from estnltk.wordnet import wn
import sqlite3
from lxml import etree

In [128]:
xml_sset_db = 'wordnet/data/xml_synsets.db'
xml_relation_db = 'wordnet/data/xml_relations.db'
rel_db = 'wordnet/data/relations_complete.db'


In [54]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS sset_relations(

                                        start_id TEXT NOT NULL,
                                        start_estwn TEXT NOT NULL,
                                        end_estwn TEXT NOT NULL,
                                        end_id TEXT NOT NULL,
                                        relation TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [129]:


conn_rel = create_connection(rel_db)
crsr_rel   = conn_rel.cursor()
    
crsr_rel.execute("ATTACH DATABASE 'wordnet/data/xml_synsets.db' AS xml_synsets ")
crsr_rel.execute("ATTACH DATABASE 'wordnet/data/xml_relations.db' AS rel_table")

In [123]:
join_str = '''CREATE TABLE synset_relations 
            AS
            SELECT
                start_index.id AS start_vertex,
                start_index.sourceSense AS start_synset, rel_table.start_estwn,
                end_index.id AS end_vertex,
                end_index.sourceSense AS end_synset, rel_table.end_estwn,
                rel_table.relation AS relation 
            FROM rel_table
            LEFT JOIN
            (
                SELECT DISTINCT id, estwn_id, sourceSense FROM xml_synsets
            ) AS start_index
            ON rel_table.start_estwn = start_index.estwn_id
            LEFT JOIN
            (
                SELECT DISTINCT id, estwn_id, sourceSense FROM xml_synsets
            ) AS end_index
            ON rel_table.end_estwn = end_index.estwn_id
'''

In [130]:
with conn_rel:
    crsr_rel.execute(join_str)    

OperationalError: table synset_relations already exists